## Наивный Байесовский классификатор

Разберём один из самых простых методов классификации &mdash; наивный байесовский классификатор. Несмотря на его простоту в некоторых задачах он работает даже лучше других, более сложных моделей. В любом случае, наивный байесовский классификатор содержит в себе важные теоретические идеи, поэтому с ним в любом случае полезно ознакомиться. 

### Применение при детекции спама.

Данные для решения задачи детекции спама можно сделать следующим образом:
1. Взять набор размеченных текстовых сообщений, часть которых размечена как спам, а остальные &mdash; как не спам;
2. Зафиксировать словарь, например, взяв все слова, встречающиеся в наборе текстовых сообщений;
3. Преобразовать текстовые данные в целочисленные, посчитав для каждого слова из словаря, встречается ли оно в данном сообщении.

Таким образом, каждому предложению соответствует вектор из нулей и единиц длины, равной мощности словаря. На полученных данных уже стандартным образом можно обучить наивный байесовский классификатор.

При реализации класса для наивного байесовского классификатора надо помнить один очень важный на практике момент: произведение вероятностей большого количества чисел может очень быстро сравняться с нулем при вычислении на компьютере, так как компьютеру может не хватить вычислительной точности. Поэтому при реализации стоит использовать **логарифмы вероятностей**.

Применим наивный байесовский классификатор к конкретному датасету https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

**1. Чтение данных.**

In [2]:
labels = []
messages = []

with open('SMSSpamCollection.txt', 'r') as fin:
    for line in fin:
        cur_label, cur_message = line.split('\t')
        labels.append(cur_label)
        messages.append(cur_message)

In [3]:
raw_df = pd.DataFrame()
raw_df['data'] = messages
raw_df['label'] = labels
raw_df.head()

,data,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...\n,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


В датасете метки бывают 2 видов: 
* `ham` &mdash; означает, что сообщение **не является спамом**,
* `spam` &mdash; означает, что сообщение **является спамом**.

**2. Предобработка данных.**

Очевидно, что сразу в таком виде нельзя передавать данные наивному байесовскому классификатору. Их надо привести к численному виду. 

Столбец `label` привести к численному виду можно очень просто.

In [4]:
raw_df['label'] = (raw_df['label'] == 'spam') * 1
raw_df.head()

,data,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


Для преобразования текстовых сообщений воспользуемся `CountVectorizer`, работающему по принципу мешка слов (*bag of words*). Он имеет следующие гиперпараметры:

* `max_df` &mdash; максимальная доля сообщений, в которых может встречатся слово из словаря, то есть в словарь не включаются слишком **частые** слова, что помогает бороться со стоп-словами;
* `min_df` &mdash; минимальная доля сообщений, в которых может встречатся слово из словаря, то есть в словарь не включаются слишком **редкие** слова;
* `max_features` &mdash; максимальное возможное количество выбранных слов, они выбираются среди наиболее частых;
* `stop_words` &mdash; можно просто взять и задать стоп-слова, которые не будут добавлены в словарь ни при каких обстоятельствах.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
vectorizer = CountVectorizer(min_df=0.03)
transformed_data = vectorizer.fit_transform(messages).toarray()

Напечатаем весь мешок слов и их количество.

In [7]:
print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names())

66
['all', 'am', 'and', 'are', 'at', 'be', 'but', 'call', 'can', 'come', 'day', 'do', 'for', 'free', 'from', 'get', 'go', 'good', 'got', 'gt', 'have', 'he', 'how', 'if', 'in', 'is', 'it', 'its', 'just', 'know', 'like', 'll', 'love', 'lt', 'me', 'my', 'no', 'not', 'now', 'of', 'ok', 'on', 'only', 'or', 'out', 'send', 'so', 'text', 'that', 'the', 'then', 'there', 'this', 'time', 'to', 'up', 'ur', 'want', 'was', 'we', 'what', 'when', 'will', 'with', 'you', 'your']


Как видим, сообщения автоматически были порезаны на слова, а слова переведены в нижний регистр.

Посмотрим на преобразованные данные.

In [8]:
print(transformed_data[:10])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0
  1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 2 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 2 1
  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0

**3. Классификатор**

В библиотеке `sklearn` имеются следующие реализации наивного байесовского классификатора:

1. `BernoulliNB` &mdash; байесовский классификатор для данных, в которых все признаки являются бинарными;
2. `MultinomialNB` &mdash; байесовский классификатор для данных, в которых все признаки являются дискретными;
3. `GaussianNB` &mdash; байесовский классификатор для вещественных данных, каждый из признаков которых имеет нормальное распределение.

Первые два метода имеют следующие параметры:
* `alpha` &mdash; коэффициент сглаживания Лапласа или Линдсона, при фиксированном значении `alpha` условные плотности будут записаны следующим образом:
$$P(X_j=x_j|Y=y) = \frac{\#\{ i : Y_i = y \text{ & } X_{ij} = x_j\} + \alpha}{\#\{i: Y_i = y\} + \alpha k_j},$$
    где $k_j$ &mdash; количество различных значений признака $x_j$; при `alpha=0` сглаживания не происходит и получаются стандартные формулы для условных вероятностей; 
* `class_prior` &mdash; арпиорные вероятности принадлежности каждому из классов;
* `fit_prior` &mdash; булевский параметр автоматического подбора априорных вероятностей принадлежности классам на основании данных.

Если ни `class_prior`, ни `fit_prior` не указаны, то априорное распределение принимается равномерным.

Гауссовский наивный байесовский классификатор предполагает распределение при фиксированном классе гауссовским:
$$P(X_i = x_i | Y = y) = \frac{1}{\sqrt{2\pi \sigma_y^2}} \cdot \exp \left( -\frac{(x_i - \mu_y)^2}{2\sigma_y^2} \right),$$
где $\mu_y$ и $\sigma_y$ оцениваются методом максимального правдоподобия.

В нашей текущей задаче для признаков, описывающих количество вхождений каждого слова из словаря в сообщение, логично использовать `MultinomialNB`. Однако после мы сравним точность предсказаний `MultinomialNB` с точностью предсказаний `BernoulliNB` для бинарных признаков: каждый признак является индикатором того, присутствует ли данное слово из словаря в сообщении.

In [9]:
from sklearn.naive_bayes import MultinomialNB

multinomial_nb = MultinomialNB()

Еще раз посмотрим на данные

In [10]:
transformed_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Как обычно, разделим данные на обучающую выборку и на тестовую.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    transformed_data, raw_df['label'], random_state=42
)

Обучаем модель и смотрим качество на тестовой выборке.

In [12]:
multinomial_nb.fit(X_train, y_train)
predictions = multinomial_nb.predict(X_test)

print(f'accuracy: {accuracy_score(y_test, predictions) :.3}')
print(f'f1 score: {f1_score(y_test, predictions) :.3}')

accuracy: 0.944
f1 score: 0.785


Результат получился весьма неплохой.

А теперь посмотрим, как с этой же задачей справится наивный байесовский классификатор на бинарных данных.

In [13]:
X_train = (X_train > 0) * 1
X_test = (X_test > 0) * 1

In [14]:
from sklearn.naive_bayes import BernoulliNB

bernoulli_nb = BernoulliNB()

In [15]:
bernoulli_nb.fit(X_train, y_train)
predictions = bernoulli_nb.predict(X_test)

print(f'accuracy: {accuracy_score(y_test, predictions) :.3}')
print(f'f1 score: {f1_score(y_test, predictions) :.3}')

accuracy: 0.95
f1 score: 0.806


**Вывод.**

Результат получился достаточно неожиданный. Наивный байесовский классификатор, обученный на бинаризованных данных показал более высокую точность классификации.

**4. Больший размер словаря**

А теперь посмотрим, что будет, если мы возьмём другое количество слов для словаря.

It's gonna be huge!

In [16]:
huge_vectorizer = CountVectorizer()
huge_data = huge_vectorizer.fit_transform(messages).toarray()
print(huge_data.shape)

(5574, 8713)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    huge_data, raw_df['label'], random_state=42
)

In [18]:
multinomial_nb.fit(X_train, y_train)
predictions = multinomial_nb.predict(X_test)

print(f'accuracy: {accuracy_score(y_test, predictions) :.3}')
print(f'f1 score: {f1_score(y_test, predictions) :.3}')

accuracy: 0.985
f1 score: 0.945


In [19]:
bernoulli_nb.fit(X_train, y_train)
predictions = bernoulli_nb.predict(X_test)

print(f'accuracy: {accuracy_score(y_test, predictions) :.3}')
print(f'f1 score: {f1_score(y_test, predictions) :.3}')

accuracy: 0.981
f1 score: 0.925


**Вывод.**

От увеличения количества рассматриваемых слов в данном случае точность предсказаний возрасла как для наивного байесовского классификатора над категориальными признаками, так и для классификатора над бинарными признаками.